# Extraindo estruturas do PubChem usando pubchempy

Neste notebook, vamos explorar o módulo `pubchempy`, que possibilita o acesso ao banco de dados PubChem usando código Python, o que nos permite converter nomes de compostos químicos (ou outros identificadores) em suas respectivas estruturas.

Para usar o `pubchempy`, é necessário instalar o módulo usando `pip`. Você pode fazer isso diretamente no seu computador digitando `pip install pubchempy` na linha de código ou, se estiver usando esse Notebook, pode rodar a célula abaixo:

In [ ]:
!pip install pubchempy

In [2]:
import pubchempy
from pubchempy import Compound, get_compounds

# Vamos importar outros módulos úteis
from rdkit import Chem
import pandas as pd

Vamos ver alguns exemplos de como extrair estruturas do PubChem:

In [3]:
# A partir do nome em inglês (glucose)
compound = get_compounds('glucose', 'name')
print(compound)

[Compound(5793)]


In [4]:
# O valor retornado foi uma lista. Para acessar os SMILES:
print(compound[0].isomeric_smiles)

C([C@@H]1[C@H]([C@@H]([C@H](C(O1)O)O)O)O)O


In [5]:
# Também é possível acessar SMILES canônicos
print(compound[0].canonical_smiles)

C(C1C(C(C(C(O1)O)O)O)O)O


In [6]:
# A partir do identificador CID
compound = pubchempy.Compound.from_cid(5090)
print(compound)

Compound(5090)


Note que nesse caso, apenas o Compound foi retornado, e não uma lista. Para obter o SMILES:

In [7]:
print(compound.isomeric_smiles)

CS(=O)(=O)C1=CC=C(C=C1)C2=C(C(=O)OC2)C3=CC=CC=C3


## Exemplo - dataset de antibióticos

Finalmente, vou apresentar um exemplo prático. Extraí alguns nomes de antibióticos e suas classes a partir da página da Wikipedia [List of antibiotics]( https://en.wikipedia.org/wiki/List_of_antibiotics) e usando o ChatGPT.

Converti os dados a um arquivo csv com os nomes dos antibióticos e suas classes:

In [8]:
df_antibiotics = pd.read_csv('../datasets/antibiotics.csv', sep=',')
df_antibiotics.head()

,name,class,SMILES
0,Amikacin,aminoglycosides,NaN
1,Gentamicin,aminoglycosides,NaN
2,Kanamycin,aminoglycosides,NaN
3,Neomycin,aminoglycosides,NaN
4,Netilmicin,aminoglycosides,NaN


Agora, vou usar o `pubchempy` para obter as estruturas no formato SMILES a partir dos nomes dos antibióticos.

Vou começar escrevendo uma função que retorna os SMILES (padronizados pelo RDKit) a partir do nome e usar o método `apply` do `pandas` para inserir os valores na coluna SMILES.

In [9]:
def smiles_from_name(name):
    compound = get_compounds(name, 'name')
    if compound == []:
        return None
    if type(compound) == list:
        isomeric_smiles = compound[0].isomeric_smiles
    else:
        isomeric_smiles = compound.isomeric_smiles
    # Padronizar SMILES com o RDKit
    try:
        rdkit_smiles = Chem.MolToSmiles(Chem.MolFromSmiles(isomeric_smiles))
    except TypeError:
        return None
    return rdkit_smiles

In [10]:
df_antibiotics['SMILES'] = df_antibiotics['name'].apply(smiles_from_name)

In [11]:
df_antibiotics.head(10)

,name,class,SMILES
0,Amikacin,aminoglycosides,NCC[C@H](O)C(=O)N[C@@H]1C[C@H](N)[C@@H](O[C@H]...
1,Gentamicin,aminoglycosides,CNC(C)C1CCC(N)C(OC2C(N)CC(N)C(OC3OCC(C)(O)C(NC...
2,Kanamycin,aminoglycosides,NC[C@H]1O[C@H](O[C@@H]2[C@@H](N)C[C@@H](N)[C@H...
3,Neomycin,aminoglycosides,NC[C@@H]1O[C@H](O[C@@H]2[C@@H](CO)O[C@@H](O[C@...
4,Netilmicin,aminoglycosides,CCN[C@@H]1C[C@H](N)[C@@H](O[C@H]2OC(CN)=CC[C@H...
5,Paromomycin,aminoglycosides,NC[C@@H]1O[C@H](O[C@@H]2[C@@H](CO)O[C@@H](O[C@...
6,Spectinomycin,aminoglycosides,CN[C@H]1[C@@H](O)[C@@H](NC)[C@H](O)[C@H]2O[C@@...
7,Streptomycin,aminoglycosides,CN[C@@H]1[C@H](O[C@H]2[C@H](O[C@@H]3[C@@H](N=C...
8,Tobramycin,aminoglycosides,NC[C@H]1O[C@H](O[C@@H]2[C@@H](N)C[C@@H](N)[C@H...
9,Geldanamycin,ansamycins,COC1=C2C[C@@H](C)C[C@H](OC)[C@H](O)[C@@H](C)/C...


Verificando se algum composto ficou sem SMILES:

In [12]:
df_antibiotics[df_antibiotics['SMILES'].isna()]

,name,class,SMILES
102,Bacitracin,polypeptides,None
104,Polymixin B,polypeptides,None
129,Sulfanilimide,sulfonamides,None
132,Sulfonamidochrysoidine,sulfonamides,None


Após investigar por que esses quatro compostos não foram encontrados no Pubchem, descobri o seguinte:

- Bacitracin: está identificado no Pubchem como Bacitracin A
- Polymixin B: é uma mistura, e não uma substância pura
- Sulfanilimide: erro de digitação, o correto é Sulfanilamide
- Sulfonamidochrysoidine: está identificada no Pubchem como Prontosil

Esses são alguns exemplos de erros comuns do que pode acontecer ao converter nomes a estruturas químicas.

Optei por remover o Polymixin B do dataset e ajustar os nomes dos outros manualmente, obtendo em seguida seus SMILES usando o `pubchempy`.

In [13]:
# Bacitracin
df_antibiotics.at[102,'name'] = 'Bacitracin A'
df_antibiotics.at[102,'SMILES'] = smiles_from_name('Bacitracin A')

In [14]:
# Sulfanilimide
df_antibiotics.at[129,'name'] = 'Sulfanilamide'
df_antibiotics.at[129,'SMILES'] = smiles_from_name('Sulfanilamide')

In [15]:
# Sulfonamidochrysoidine
df_antibiotics.at[132,'name'] = 'Prontosil'
df_antibiotics.at[132,'SMILES'] = smiles_from_name('Prontosil')

In [16]:
# Remover Polymixin B
df_antibiotics = df_antibiotics.drop(104)

In [17]:
# Confirmar que todas as linhas têm SMILES
df_antibiotics[df_antibiotics['SMILES'].isna()]

,name,class,SMILES


Finalmente, vou exportar o *dataset* no formato csv. Você pode encontrá-lo na pasta `datasets` no GitHub.

In [18]:
# Exportar dataset com os SMILES
df_antibiotics.to_csv('../datasets/antibiotics_SMILES.csv', sep=',', index=False, index_label=False)